In [37]:
import pandas as pd
import pyodbc
import urllib

# Installing the FuzzyWuzzy Algo
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from sqlalchemy import event
from sqlalchemy import create_engine

# Specify the connection details
server = '172.16.3.22'
database = 'patentview_trial'
username = 'Tipu_Thakur'
password = 'IDSL2Sb!gt%2'

# Create a connection string
connection_string = f"Driver={{SQL Server}};Server={server};Database={database};UID={username};PWD={password};"

# Establish the database connection
connection = pyodbc.connect(connection_string)



In [38]:
# Create a cursor object
cursor = connection.cursor()

# Execute the first SQL query
query1 = """
SELECT REC_ID,paper_id,patent_id,dcterms_creator,inventor_id,Author_Name_Corrected,Inventor_Name_Corrected,CORRECTED_NAME_DISTANCE,ORIGINAL_NAME_DISTANCE
FROM [dbo].DERV_PAPER_PATENT_LINKAGE_DISTINCT
"""

cursor.execute(query1)


In [5]:

# Fetch the first query results
results1 = cursor.fetchall()

# Get the column names from the cursor description
columns1 = [column[0] for column in cursor.description]

# Convert the first query results to a DataFrame
df1 = pd.DataFrame.from_records(results1, columns=columns1)


In [6]:
# Define a fuzzy match function based on thresholds

def fuzzy_match(string1, string2, threshold):
    similarity_score = fuzz.ratio(string1, string2)
    if similarity_score >= threshold:
        return similarity_score
    else:
        return 0

In [9]:
df1['similarity_score_50'] = df1.apply(lambda x: fuzzy_match(x['Author_Name_Corrected'], x['Inventor_Name_Corrected'], 50), axis=1)
df1['similarity_score_60'] = df1.apply(lambda x: fuzzy_match(x['Author_Name_Corrected'], x['Inventor_Name_Corrected'], 60), axis=1)
df1['similarity_score_70'] = df1.apply(lambda x: fuzzy_match(x['Author_Name_Corrected'], x['Inventor_Name_Corrected'], 70), axis=1)
df1['similarity_score_80'] = df1.apply(lambda x: fuzzy_match(x['Author_Name_Corrected'], x['Inventor_Name_Corrected'], 80), axis=1)
df1['similarity_score_90'] = df1.apply(lambda x: fuzzy_match(x['Author_Name_Corrected'], x['Inventor_Name_Corrected'], 90), axis=1)
df1['similarity_score_95'] = df1.apply(lambda x: fuzzy_match(x['Author_Name_Corrected'], x['Inventor_Name_Corrected'], 95), axis=1)

In [12]:
# Define a Jaccard Token Similarity Function
def token_jaccard_similarity(string1, string2):
    set1 = set(string1.split())
    set2 = set(string2.split())
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    jaccard_similarity = len(intersection) / len(union)
    return jaccard_similarity

In [13]:
df1['token_jaccard_similarity'] = df1.apply(lambda x: token_jaccard_similarity(x['Author_Name_Corrected'], x['Inventor_Name_Corrected']), axis=1)

In [18]:
# Export the DataFrame to a file with a delimiter
df1.to_csv('E:\Sagar\EMAKG\Paper_Patent_Linkage\Scoring.csv', sep='|', index=False)

In [4]:
# Load the DataFrame from File
df1 = pd.read_csv('E:\Sagar\EMAKG\Paper_Patent_Linkage\Scoring.csv', sep='|')

In [54]:
df1.head()

,REC_ID,paper_id,patent_id,dcterms_creator,inventor_id,Author_Name_Corrected,Inventor_Name_Corrected,CORRECTED_NAME_DISTANCE,ORIGINAL_NAME_DISTANCE,similarity_score_50,similarity_score_60,similarity_score_70,similarity_score_80,similarity_score_90,similarity_score_95,token_jaccard_similarity
0,182,3034306036,7183352,1970217858,fl:ra_ln:pires-1,ROBERT REYER,RAUL PIRES,9,9,0,0,0,0,0,0,0.000000
1,288,3033631694,10651815,2293872643,fl:we_ln:huang-150,WEI CHUNG HSU,WEI HUANG,6,7,73,73,73,0,0,0,0.250000
2,362,3033346716,5688795,2226900775,fl:da_ln:clarke-4,DAVID J MORGANS,DAVID E CLARKE,8,8,55,0,0,0,0,0,0.200000
3,465,3031618475,5660813,3028934885,fl:ag_ln:fogo-1,FOGO AGNES,AGNES FOGO,10,10,50,0,0,0,0,0,1.000000
4,517,3031365499,7022684,2225223891,fl:mi_ln:pacey-1,MICHAEL STEPHEN PACEY,MICHAEL PACEY,8,8,76,76,76,0,0,0,0.666667


In [59]:
# Loading the File to SQL Table

params = 'DRIVER='+'ODBC Driver 17 for SQL Server' + ';SERVER='+server + ';PORT=1433;DATABASE=' + database + ';UID=' + username + ';PWD=' + password
cnxn = pyodbc.connect(params)
# Create a cursor object
insert_cursor = cnxn.cursor()

db_params = urllib.parse.quote_plus(params)

engine = create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params))

df1.to_sql("DERV_PAPER_PATENT_LINKAGE_SIMILARITY", engine, if_exists='append', index=False, schema="dbo")


-1